# XML Information Dataset Exploration

We're going to take a look at a few examples of how we can explore the XML Information dataset.

In [ ]:
dataset = "ARCHDATASETURL"

## pandas

Next, we'll setup our environment so we can load our XML Information dataset into [pandas](https://pandas.pydata.org) DataFrames. If you're unfamiliar with DataFrames, but you've worked with spreadsheets before, you should feel comfortable pretty quick.

In [ ]:
import pandas as pd

## Data Table Display

Colab includes an extension that renders pandas DataFrames into interactive displays that can be filtered, sorted, and explored dynamically. This can be very useful for taking a look at what each DataFrame provides, and doing some intital filtering!

Data table display for pandas DataFrames can be enabled by running:
```python
%load_ext google.colab.data_table
```
and disabled by running
```python
%unload_ext google.colab.data_table
```

In [ ]:
%load_ext google.colab.data_table

## Loading our ARCH Dataset as a DataFrame

---


Next, we'll create pandas DataFrame from our dataset, and show a preview of it using the Data Table Display.

In [ ]:
xml = pd.read_csv(dataset, compression="gzip", skipinitialspace=True)
xml

# Data Analysis

Now that we have all of our datasets loaded up, we can begin to work with them!

## Counting total files, and unique files

Let's take a quick look at how to count items in DataFrames, and use total and unique files as an example to work with.

It's definitely work checking out the [pandas documentation](https://pandas.pydata.org/docs/index.html). There are a lot of good examples available, along with a robust [API reference](https://pandas.pydata.org/docs/reference/index.html#api).


### How many XML files are in this collection?

We can take our `xml` variable and try a couple of functions to get the same answer.

1.   `len(xml.index)`
  * Get the length of the DataFrame's index.
2.   `xml.shape[0]`
  * Get the shape or dimensionality of the DataFrame, and take the first item in the tuple.
3.  `xml.count()`
  * Count the number of rows for each column.



In [ ]:
len(xml.index)

In [ ]:
xml.shape[0]

In [ ]:
xml.count()

### How many unique XML files are in the collection?

 We can see if an XML file is unique or not by computing an [MD5 hash](https://en.wikipedia.org/wiki/MD5#MD5_hashes) of it, and comparing them. The exact same XML file might have a filename of `example.svg` or `foo.svg`. If the hash is computed for each, we can see that even with different file names, they are actually the same XML file. So, since we have both a `MD5` and `SHA1` hash column available in our DataFrame, we can just find the unique values, and count them!




In [ ]:
len(xml.md5.unique())

### What are the top 10 most occurring xml files in the collection?

Here we can take advantage of [`value_counts()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html) to provide us with a list of MD5 hashes and their respective counts.

In [ ]:
xml["md5"].value_counts().head(10)

Get the top value as a variable using [`mode()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html).

In [ ]:
most_occurring = xml["md5"].mode()[0]
most_occurring

### What's the information around all the most occurring file?

Let's find those xml files in the DataFrame. We can here see some of the filenames used, MIME types, and its URL.

In [ ]:
xml.loc[xml["md5"] == most_occurring]

### What are the top 10 most occurring filenames in the collection?

Note that this is of course different than the MD5 results up above. Here we are focusing _just_ on filename. So `index.xml` for example, might actually be referring to different xml files who happen to have the same name.

Here we can use `value_counts()` again, but this time we'll create a variable for the top filenames so we can use it later.



In [ ]:
top_filenames = xml["filename"].value_counts().head(10)
top_filenames

### Let's create our first graph!

We'll first plot the data with the pandas [plot](https://pandas.pydata.org/docs/reference/api/pandas.Series.plot.html) functionality, and then with [Altair](https://altair-viz.github.io/).

In [ ]:
top_filenames_chart = top_filenames.plot.bar(figsize=(25, 10))

top_filenames_chart.set_title("Top Filenames", fontsize=22)
top_filenames_chart.set_xlabel("Filename", fontsize=20)
top_filenames_chart.set_ylabel("Count", fontsize=20)

Now let's setup [Altair](https://altair-viz.github.io/), and plot the data. Altair is useful for creating vizualizations since they can be easily exported as a PNG or SVG.

In [ ]:
import altair as alt

In [ ]:
top_filenames_altair = (
    xml["filename"]
    .value_counts()
    .head(10)
    .rename_axis("Filename")
    .reset_index(name="Count")
)

filenames_bar = (
    alt.Chart(top_filenames_altair)
    .mark_bar()
    .encode(x=alt.X("Filename:O", sort="-y"), y=alt.Y("Count:Q"))
)

filenames_rule = (
    alt.Chart(top_filenames_altair).mark_rule(color="red").encode(y="mean(Count):Q")
)


filenames_text = filenames_bar.mark_text(align="center", baseline="bottom").encode(
    text="Count:Q"
)

(filenames_bar + filenames_rule + filenames_text).properties(
    width=1400, height=700, title="Top Filenames"
)

### How about a file format distribution?

What _kind_ of XML files are present? We can discover this by checking their "media type", or [MIME type](https://en.wikipedia.org/wiki/Media_type). 






In [ ]:
xml_mime_types = (
    xml["mime_type_tika"]
    .value_counts()
    .head(5)
    .rename_axis("MIME Type")
    .reset_index(name="Count")
)

xml_mimes_bar = (
    alt.Chart(xml_mime_types)
    .mark_bar()
    .encode(x=alt.X("MIME Type:O", sort="-y"), y=alt.Y("Count:Q"))
)

xml_mime_rule = (
    alt.Chart(xml_mime_types).mark_rule(color="red").encode(y="mean(Count):Q")
)

xml_mime_text = xml_mimes_bar.mark_text(align="center", baseline="bottom").encode(
    text="Count:Q"
)

(xml_mimes_bar + xml_mime_rule + xml_mime_text).properties(
    width=1400, height=700, title="XML File Format Distribution"
)

### How do I get the actual XML files?

...or, how do I get to the actual binary files described by each file format information derivative?

There are a few options!

1. `wget` or `curl` from the live URL, or a replay URL
  * Live web URL
    * `wget` or `curl` the value of the `url` column
  * Replay web URL
    * `wget` or `curl` the value of the `crawl_date` and `url` column using the following pattern:
      * `https://web.archive.org/web/` + `crawl_date` + `/` + `url`
        * https://web.archive.org/web/20120119124734/http://www.archive.org/images/glogo.png
      * `http://wayback.archive-it.org/14462/` + `crawl_date` + `/` + `url`
        * https://wayback.archive-it.org/14462/20210524212740/https://ruebot.net/visualization/elxn42/featured_hu33a17dfb90e2c5ed77f783db14a6e53a_5126291_550x0_resize_q90_box_2.png
2. Use a scripting language, such as Python
  * Make use of the `url` and `filename` columns (and `crawl_date` if you want to use the replay URL)
  * `import requests`
  * `requests.get(url, allow_redirects=True)`
  * `open('filename', 'wb').write(r.content)`
3. Use the [Archives Unleashed Toolkit](https://aut.docs.archivesunleashed.org/docs/extract-binary) (if you have access to the W/ARC files).

If you wanted to download the XML files using the replay URL, below is a method for doing so.

First, you'll want to setup a replay url base url. Here we'll use the Archive-It Wayback instance for the collection.

In [ ]:
wayback_url = "ARCHCOLLECTIONIDURL"

Next we'll create a new column using a lambda function. If you're familiar working with spreadsheets, what we're doing here is basically concatenating some column values together and creating a new column.

In [ ]:
xml["replay_url"] = xml.apply(
    lambda row: str(wayback_url + str(row["crawl_date"]) + "/" + row["url"]), axis=1
)

Then we can export that new column we created out to a file, so we can use it with `wget` to download all the XML files!

In [ ]:
xml["replay_url"].head().to_csv("xml_urls.txt", index=False, header=False)

Finally, we can pass the file to `wget` to use as a download list. You can also speed this process up using `xargs` or `parallel`.

In [ ]:
!wget --random-wait -i xml_urls.txt